# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the FEP package.
  * Set things up to carry out geometry optimization using Amber.

In [1]:
(ql:quickload :fep)

To load "fep":
  Load 1 ASDF system:
    fep

; Loading "fep"
.......   build-fasl seconds real(5.8) run(5.8) llvm(0) link(5.6) (llvm+link)/real(96%)
..   build-fasl seconds real(1.2) run(1.2) llvm(0) link(1.1) (llvm+link)/real(96%)
.......   build-fasl seconds real(6.4) run(6.4) llvm(0) link(6.1) (llvm+link)/real(95%)
..   build-fasl seconds real(1.3) run(1.3) llvm(0) link(1.2) (llvm+link)/real(96%)



(:FEP)

In [ ]:
(leap:setup-amber-paths)

In [ ]:
(source "leaprc.protein.ff14SB")

# Start a new FEP calculation

In [ ]:
(:= *feps* (make-instance 'fep:calculation))

# Import the sketch from a Chemdraw file

In [ ]:
(png-from-file "ligand2.png" )

In [ ]:
(:= *sketch* (fep:load-chem-draw-fep "ligand2.cdxml"))

In [ ]:
(fep:setup-ligands *feps* *sketch*)

In [ ]:
(:= v (show (fep:layout-ligands *feps*)))
v

# Load the Thrombin PDB file

In [ ]:
(:= *thrombin* (load-pdb "2zff_fixed.pdb"))

In [ ]:
(:= vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(nglv::add-representation vthrombin "ball+stick" :selection "all")

In [ ]:
(remove-molecules *thrombin* 'solvent)

In [ ]:
(chem:setf-molecule-type (chem:content-with-name *thrombin* :|53U|) 'ligand)

In [ ]:
(push *thrombin* (fep:receptors *feps*))

# Load a template ligand

In [ ]:
(:= *lig* (load-pdb "ligand.pdb"))

In [ ]:
(show *lig*)

In [ ]:
(defmacro named-test (name)
    `(cons ,name (lambda (a) (eq (chem:get-name a) ,name))))

Define a SMARTS matcher that labels ligand atoms

In [ ]:
(:= *tests* (mapcar (lambda (name) (named-test name)) '(:c13 :c12 :c15 :c9 :c8 :c7 :c5 :c3 :c1 :c20 :c18 :n1 :n3 :o2 :o1 :c2)))
(:= *pick* (chem:compile-smarts 
             "[C<c13>:13]1~[C<c12>:12]~C~C(~[C<c15>:15]~C~C1)~[C<c9>:9]~[N<n3>:33]~[C<c8>:8](~[O<o2>:22])[C<c7>:7]6C[C<c5>:5]CN(C6)[C<c3>:3](~[O<o1>:21])[C<c2>:2]([N<n1>:31])[C<c1>:1]C10~C~[C<c20>:20]~C~[C<c18>:18]~C~C10" :tests *tests*))


Test the SMARTS code against the atoms of the ligand - it should recognize atom C13

In [ ]:
(cando:do-atoms (a *lig*) (when (chem:matches *pick* a)
                              (format t "Match!! ~a -> ~a~%" a (chem:matches *pick* a))))

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")
(load-amber-params "gaff.dat")

In [ ]:
(fep:build-ligands *feps*)

In [ ]:
(show (fep:layout-ligands *feps* :accessor 'fep::molecule))

# Pose the new ligands onto the template ligand

In [ ]:
(fep:pose-ligands-using-pattern *feps* *pick* *lig* :stereochemical-restraints '( (2 . :R) (7 . :S)))

In [ ]:
(:= vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(:= *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (fep::molecule (first (fep:ligands *feps*))))
(:= *struct* (make-instance 'cando-structure :matter *moveable-agg*))
(nglv::add-structure vthrombin *struct*)

In [ ]:
(fep:molecule (first (fep:ligands *feps*)))

In [ ]:
(show (fep:layout-ligands *feps* :accessor 'fep::molecule))

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(fep::build-initial-jobs *feps* :connections 2 :stages 3 :windows 3)

In [ ]:
(fep:jobs *feps*)

In [ ]:
(graph::save-graph *feps*)